In [71]:
import findspark
findspark.init('/home/thiagodeschamps/spark-3.3.1-bin-hadoop3')

In [72]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
                    .appName('resp-entregavel_2')\
                    .getOrCreate()

In [73]:
transacoes = [{'transacao_id':1, 'total_bruto':3000, 'desconto_percentual':6.99},
              {'transacao_id':2, 'total_bruto':57989, 'desconto_percentual':1.45},
              {'transacao_id':4, 'total_bruto':1, 'desconto_percentual':None},
              {'transacao_id':5, 'total_bruto':34, 'desconto_percentual':0.0}]

In [74]:
df = spark.createDataFrame(transacoes).na.fill(1) 

In [75]:
df.show()

+-------------------+-----------+------------+
|desconto_percentual|total_bruto|transacao_id|
+-------------------+-----------+------------+
|               6.99|       3000|           1|
|               1.45|      57989|           2|
|                1.0|          1|           4|
|                0.0|         34|           5|
+-------------------+-----------+------------+



In [76]:
df.printSchema()

root
 |-- desconto_percentual: double (nullable = false)
 |-- total_bruto: long (nullable = true)
 |-- transacao_id: long (nullable = true)



In [78]:
df.withColumn('Result', ( df['total_bruto'] - (df['desconto_percentual'] - 1)/100)).show()



+-------------------+-----------+------------+----------+
|desconto_percentual|total_bruto|transacao_id|    Result|
+-------------------+-----------+------------+----------+
|               6.99|       3000|           1| 2999.9401|
|               1.45|      57989|           2|57988.9955|
|                1.0|          1|           4|       1.0|
|                0.0|         34|           5|     34.01|
+-------------------+-----------+------------+----------+

